In [96]:
# Packages needed, may not be in git yet
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.constants as sc
%matplotlib inline
from eos import *
from astro_const import *
from structure import *
from reactions import *
from scipy.optimize import bisect
from astro_const import *
from zams import *

In [93]:
delta_m = 1e-80
eta = 1e-11
xi = 5e-2

Z = [1,2,7]
A = [1,4,14]
X = [.706,.275,.019]
comp = [Z,A,X]
mu = mean_molecular_weight(Z,A,X)
Pc, rhoc, Tc = central_thermal(0.1,0.1,mu) # Mass and radius are in units of solar
m, radius, pressure, Luminosity = integrate(0.1, 0.1, delta_m, eta, xi, comp, max_steps=10000, pp_factor=1.0) # Pc,rhoc,Tc,delta_m,eta,xi,mu,XH

print(m[-1],radius[-1],pressure[-1],Luminosity[-1])

1.988335266312407e+29 69568098.12354356 878396.5757287166 1.6955174148164621e+25


In [91]:
delta_m = 5e-61
eta = 1e-11
xi = 5e-2

m, radius, pressure, Luminosity = integrate(0.1, 0.1, delta_m, eta, xi, comp, max_steps=10000, pp_factor=1.0) # Pc,rhoc,Tc,delta_m,eta,xi,mu,XH

print(m[-1],radius[-1],pressure[-1],Luminosity[-1])

1.988335275005056e+29 69568064.08874863 898523.9292784276 1.6955174230785984e+25


In [97]:
def g(M,R,comp,delta_m,eta,xi):
    m,r,p,l = integrate(M,R,delta_m,eta,xi,comp,max_steps=10000,pp_factor=1.0)
    L = l[-1]
    R = r[-1]
    T_eff = Teff(Mwant)
    Lwant = surface_luminosity(T_eff,R)
    return(L-Lwant)

In [ ]:
masses = np.array([0.1,0.15,0.2,0.3])
Teffs = np.array([2800.0,3150.0,3300.0,3400.0])

for i in range(len(masses)):
    Pguesses[i] = Pguess
    Pc = bisect(f,Pguess/10,Pguess*10,args = (masses[i],delta_m,eta,xi,mue))
    Pcs[i] = Pc

In [2]:
delta_m = 1e-4
eta = 1e-11
xi = 0.01

masses = Msun*np.arange(0.1,1.1,0.1)
Pcs = np.zeros(10)

def f(Pc,Mwant,delta_m,eta,xi,mue):
    m,r,p = integrate(Pc,delta_m,eta,xi,mue)
    M = m[-1]
    return(M-Mwant)

for i in range(len(masses)):
    Pguess = pressure_guess(masses[i],mue)
    Pguesses[i] = Pguess
    Pc = bisect(f,Pguess/100,100*Pguess,args = (masses[i],delta_m,eta,xi,mue))
    Pcs[i] = Pc

central_dens = density(Pcs,mue)

NameError: name 'pressure_guess' is not defined

In [ ]:
Radii = np.zeros(10)

for i in range(len(masses)):
    m,r,p = integrate(Pcs[i],delta_m,eta,xi,mue)
    Radii[i] = r[-1]

In [ ]:
R = Radii/Rsun
Pcsnorm = Pcs/(G*(masses**2)*Radii**(-4))
central_dens_norm = central_dens/((3*masses)/(4*np.pi*Radii**(3)))

In [ ]:
for i in range(len(masses)):
    Pguess = pressure_guess(masses[i],mue)
    Pc = bisect(f,0.9*Pguess,1.1*Pguess,args = (masses[i],delta_m,eta,xi,mue))
    Pcs[i] = Pc

central_dens = density(Pcs,mue,K = 2)

Radii = np.zeros(10)

for i in range(len(masses)):
    m,r,p = integrate(Pcs[i],delta_m,eta,xi,mue)
    Radii[i] = r[-1]

In [ ]:
array = np.array([masses/Msun,R,Pcs,Pcsnorm,central_dens,central_dens_norm])
MRdf = pd.DataFrame(array,index = ["$\dfrac{M}{M_{\odot}}$","$\dfrac{R}{R_\odot}$",
                                   "$P_c$","$\dfrac{P_c}{GM^2R^{-4}}$","$\rho_c$",
                                   "$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$"]).transpose()
MRdf.columns = ["$$\dfrac{M}{M_{\odot}}$$","$$\dfrac{R}{R_\odot}$$",
                                   "$$P_c$$","$$\dfrac{P_c}{GM^2R^{-4}}$$","$$\rho_c$$",
                                   "$$\dfrac{\rho_c}{\dfrac{3M}{4\pi R^3}}$$"]
MRdf

In [5]:
!python -m pytest

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /Users/jkho/Documents/MSU Fall 2022/AST 304/Project 3/AST_Final_Project
plugins: anyio-2.2.0
collected 7 items                                                              

test_const.py .                                                          [ 14%]
test_eos.py ..                                                           [ 42%]
test_reactions.py .                                                      [ 57%]
test_structure.py .                                                      [ 71%]
test_zams.py ..                                                          [100%]

=============================== warnings summary ===============================
../../../../../opt/anaconda3/lib/python3.8/site-packages/astropy/version.py:26
  /Users/jkho/opt/anaconda3/lib/python3.8/site-packages/astropy/version.py:26: DeprecationWarning: dist

In [7]:
!python testing.py

chemistry passed testing
adiabatic EOS passed testing
reactions passed testing
central temperature passed testing
effective temperature passed testing
surface luminosity passed testing


In [94]:
def test_integration(M,r,delta_m,xi,eta):
    Rsun = ac.Rsun
    Lsun = ac.Lsun
    Z = [1,2,7]
    A = [1,4,14]
    X = [.706,.275,.019]
    XH=0.706
    mu = mean_molecular_weight(Z,A,X)
    Pc, rhoc, Tc = central_thermal(M,r,mu)
    comp = [Z,A,X]
    m_anything, radius, pressure, Luminosity = integrate(M, r, delta_m, eta, xi, comp, max_steps=10000, pp_factor=1.0) # Pc,rhoc,Tc,delta_m,eta,xi,mu,XH
    print(Tc)
    #print(m_anything[-1]/Masun, m_anything[0]/Msun)
    print(radius/Rsun)
    print(Luminosity[-1]/Lsun)
    #print(Luminosity[-1]/Lsun,Luminosity[0]/Lsun )

test_integration(.3,.333,1.0e-8,.005,10**(-12))

6851356.699730266
[0.0005901  0.00059304 0.00059599 ... 0.33299995 0.33299996 0.33299997]
0.006078146648539353
